In [1]:
!pip install tensorflow tensorflow-hub opencv-python numpy matplotlib



Defaulting to user installation because normal site-packages is not writeable
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/331.7 MB ? eta -:--:--
   ---------------------------------------- 3.1/331.7 MB 20.5 MB/s eta 0:00:17
   - -------------------------------------- 8.7/331.7 MB 23.3 MB/s eta 0:00:14
   - -------------------------------------- 13.4/331.7 MB 24.0 MB/s eta 0:00:14
   -- ------------------------------------- 18.6/331.7 MB 23.5 MB/s eta 0:00:14
   -- ------------------------------------- 23.6/331.7 MB 23.7 MB/s eta 0:00:14
   --- ------------------------------------ 28.0/331.7 MB 23.1 MB/s eta 0:00:14
   ---- ----------------------------------- 34.9/331.7 MB 24.6 MB/s eta 0:00:13
   ----- ---------------------------------- 43.3/331.7 MB 26.5 MB/s eta 0:00:11
   ------ --------------------------------- 51.4/331.7 MB 28.0 MB/s eta 0:00:11
  

In [30]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os


In [31]:
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']


In [71]:
def get_pose(frame):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   # 🔥 IMPORTANT FIX
    img = cv2.resize(img, (192,192))
    img = tf.expand_dims(img, axis=0)
    img = tf.cast(img, tf.int32)

    outputs = movenet(img)
    kp = outputs['output_0'].numpy()[0][0]
    return kp


In [72]:
prev_kp = None

def smooth_pose(kp, alpha=0.7):
    global prev_kp
    if prev_kp is None:
        prev_kp = kp
        return kp
    kp = alpha * prev_kp + (1 - alpha) * kp
    prev_kp = kp
    return kp


In [73]:
EDGES = [
    (0,1),(1,3),(0,2),(2,4),
    (0,5),(0,6),(5,7),(7,9),
    (6,8),(8,10),(5,6),
    (5,11),(6,12),(11,12),
    (11,13),(13,15),(12,14),(14,16)
]

def draw_skeleton(frame, kp):
    h, w, _ = frame.shape
    for p in kp:
        if p[2] > 0.3:
            x,y = int(p[0]*w), int(p[1]*h)
            
            cv2.circle(frame,(x,y),4,(0,255,0),-1)

    for a,b in EDGES:
        if kp[a][2] > 0.3 and kp[b][2] > 0.3:
            x1,y1 = int(kp[a][0]*w), int(kp[a][1]*h)
            x2,y2 = int(kp[b][0]*w), int(kp[b][1]*h)
            cv2.line(frame,(x1,y1),(x2,y2),(0,255,0),2)


In [74]:
def angle(a,b,c):
    ba, bc = a-b, c-b
    return np.degrees(np.arccos(
        np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc)+1e-6)
    ))


In [77]:
NOSE = 0
LS, RS = 5, 6
LH, RH = 11, 12
LK = 13
LA = 15

def extract_features(kp):
    stance = abs(kp[15][0] - kp[16][0])
    hip = abs(kp[LH][0] - kp[RH][0])
    head = kp[NOSE][0]

    knee = angle(
        kp[LH][:2],
        kp[LK][:2],
        kp[LA][:2]
    )

    swing = angle(
        kp[LS][:2],
        kp[LS][:2] + np.array([0,-1]),
        kp[9][:2]
    )

    return stance, hip, head, knee, swing



In [78]:
def extract_video_pose(video_path, max_frames=40):
    cap = cv2.VideoCapture(video_path)
    features = []

    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        kp = get_pose(frame)
        features.append(extract_features(kp))
        count += 1

    cap.release()
    return np.array(features)



In [79]:
# Load reference videos
ref1 = extract_video_pose("cricket-ai-bot/data/WhatsApp Video 2025-12-21 at 14.08.41.mp4")
ref2 = extract_video_pose("cricket-ai-bot/data/WhatsApp Video 2025-12-21 at 14.08.43.mp4")
ref3 = extract_video_pose("cricket-ai-bot/data/WhatsApp Video 2025-12-21 at 14.08.45.mp4")

# Combine all reference features
all_refs = np.vstack([ref1, ref2, ref3])

# 🔥 FINAL IDEAL PROFILE (array of 5 values)
ideal = np.mean(all_refs, axis=0)

print("✅ Ideal reference profile:", ideal)


✅ Ideal reference profile: [1.45674906e-02 1.08581818e-02 2.46211671e-01 1.35446704e+02
 4.18298584e+00]


In [80]:
def score(val, ideal, tol):
    return max(0, min(100, 100 - abs(val-ideal)/tol * 100))

def compute_scores(feat, ideal):
    stance, hip, head, knee, swing = feat
    return [
        score(stance, ideal[0], 0.15),
        score(hip, ideal[1], 0.10),
        score(knee, ideal[3], 40),
        score(swing, ideal[4], 25)
    ]



In [81]:
def draw_panel(frame, scores):
    x = frame.shape[1] - 260
    cv2.rectangle(frame,(x,0),(frame.shape[1],160),(15,15,15),-1)

    labels = ["Stance","Hip","Knee","Swing"]
    for i,(l,s) in enumerate(zip(labels,scores)):
        cv2.putText(frame,f"{l}: {int(s)}",
            (x+10,30+i*30),
            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,255),2)



In [68]:
OUTPUT_DIR = "output"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print("✅ Output folder created")
else:
    print("📁 Output folder already exists")


📁 Output folder already exists


In [82]:
# Input video path
input_video = r"cricket-ai-bot\data\\input\WhatsApp Video 2025-12-21 at 14.27.03.mp4"

# Output path
output_path = os.path.join("output", "final_output.mp4")

cap = cv2.VideoCapture(input_video)

if not cap.isOpened():
    print("❌ Input video not opened. Check path.")
    exit()

out = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*'mp4v'),
    30,
    (int(cap.get(3)), int(cap.get(4)))
)

all_scores = []   # 🔥 store frame scores

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    kp = smooth_pose(get_pose(frame))
    draw_skeleton(frame, kp)

    feat = extract_features(kp)
    scores = compute_scores(feat, ideal)

    # Store overall frame score
    all_scores.append(np.mean(scores))

    # Draw side panel
    draw_panel(frame, scores)

    out.write(frame)

cap.release()
out.release()

final_score = int(np.mean(all_scores))

print("🏏 FINAL TECHNIQUE SCORE:", final_score)
print("📁 Output saved at:", output_path)


🏏 FINAL TECHNIQUE SCORE: 76
📁 Output saved at: output\final_output.mp4
